In [2]:
!python3 -m pip install kfp --upgrade --user

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
EXPERIMENT_NAME = 'Simple notebook pipeline'
BASE_IMAGE = 'tensorflow/tensorflow:2.0.0b0-py3'

In [4]:
import json
import numpy as np
import kfp
import kfp.dsl as dsl
from kfp import compiler
from kfp import components

In [5]:
@dsl.python_component(
    name='mult_op',
    description='multiplies two matrix',
    base_image=BASE_IMAGE
)
def matrix_mult(A_: str, B_: str) -> str:
    import json
    import numpy as np
    A, B = json.loads(A_), json.loads(B_)
    C = np.matmul(A, B).tolist()
    print(f"{A}\n@\n{B}\n=\n{C}")
    return json.dumps(C)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) python_component. (This decorator does not seem to be used, so we deprecate it. If you need this decorator, please create an issue at https://github.com/kubeflow/pipelines/issues) -- Deprecated since version 0.2.6.
  after removing the cwd from sys.path.


In [6]:
mult_op = components.func_to_container_op(
    matrix_mult, 
    base_image=BASE_IMAGE
)

In [7]:
@dsl.pipeline(
   name='My pipeline',
   description='A pipeline for matrix multiplication.'
)
def calc_pipeline(
A, B
):
    task1 = mult_op(A, B)
    task2 = mult_op(task1.output, B)

In [8]:
pipeline_func = calc_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
compiler.Compiler().compile(pipeline_func, pipeline_filename)

In [9]:
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

In [10]:
arguments = {'A': json.dumps([[1,2], [4,5]]), 'B': json.dumps([[1,0], [0,1]])}

run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)